In [1]:
import time
import sys
import os
import numpy as np
import networkx as nx
import random
import time
from matplotlib import pyplot as plt

sys.path.insert(0, os.path.join(os.getcwd(), "..", "cpp_prim", "x64", "Release"))
sys.path.insert(0, os.path.join(os.getcwd(), "..", "py_prim"))
import py_prim
import cpp_prim

In [33]:
def generate_complete_graph(nodes):
    graph = nx.complete_graph(nodes)
    for (start, end) in graph.edges:
        graph.edges[start, end]['weight'] = random.randrange(1, 100)
    return graph

def generate_random_tree(nodes):
    graph = nx.generators.trees.random_tree(nodes)
    for (start, end) in graph.edges:
        graph.edges[start, end]['weight'] = random.randrange(1, 100)
    return graph

def extract_adjacency_matrix(nx_graph):
    adjacency_matrix = nx.to_numpy_array(nx_graph, dtype='int32').tolist()
    return adjacency_matrix

In [41]:
def test_on_complete_graph(prim_impl, nodes_range = 300, step = 10):
    elapsed_time = []
    nodes_num = []
    for nodes in list(range(2, nodes_range, step)):
        print(f"Test on complete graph with {nodes} nodes.", end="\r")
        nodes_num.append(nodes)
        graph = generate_complete_graph(nodes)
        adjacency_matrix = extract_adjacency_matrix(graph)
        start_time = time.time()
        prim_impl.run_algorithm(adjacency_matrix)
        elapsed_time.append(time.time() - start_time)
    return elapsed_time, nodes_num

def test_on_tree(prim_impl, nodes_range = 300, step = 10):
    elapsed_time = []
    nodes_num = []
    for nodes in list(range(2, nodes_range, step)):
        print(f"Test on random tree with {nodes} nodes.", end="\r")
        nodes_num.append(nodes)
        graph = generate_random_tree(nodes)
        adjacency_matrix = extract_adjacency_matrix(graph)
        start_time = time.time()
        prim_impl.run_algorithm(adjacency_matrix)
        elapsed_time.append(time.time() - start_time)
    return elapsed_time, nodes_num

In [49]:
def plot_timing(ax, cpp_time=None, cpp_nodes=None, py_time=None, py_nodes=None, title=""):
    if cpp_time:
        ax.plot(cpp_nodes, cpp_time, label='C++')
    if py_time:
        ax.plot(py_nodes, py_time, label='Python')
    ax.set_xlabel('Number of nodes')
    ax.set_ylabel('Time')
    ax.set_title(title)
    ax.legend()

## Complete graph

In [58]:
cpp_complete_time, cpp_complete_nodes = test_on_complete_graph(cpp_prim, nodes_range=300, step=10)

In [60]:
py_complete_time, py_complete_nodes = test_on_complete_graph(py_prim, nodes_range=1800, step=60)

In [ ]:
fig, ax = plt.subplots(1, 1)
cpp_fig, cpp_ax = plt.subplots(1, 1)
py_fig, py_ax = plt.subplots(1, 1)
plot_timing(ax, cpp_time=cpp_complete_time, 
            cpp_nodes=cpp_complete_nodes, 
            py_time=py_complete_time,
            py_nodes=py_complete_nodes,
            title="Complete grap")
plot_timing(cpp_ax, cpp_time=cpp_tree_time, cpp_nodes=cpp_tree_nodes, title="Complete graph")
plot_timing(py_ax, py_time=py_tree_time, py_nodes=py_tree_nodes, title="Complete graph")

## Random tree

In [64]:
cpp_tree_time, cpp_tree_nodes = test_on_tree(cpp_prim, nodes_range=300, step=5)

In [ ]:
py_tree_time, py_tree_nodes = test_on_tree(py_prim, nodes_range=15000, step=250)

In [ ]:
fig, ax = plt.subplots(1, 1)
cpp_fig, cpp_ax = plt.subplots(1, 1)
py_fig, py_ax = plt.subplots(1, 1)
plot_timing(ax, cpp_time=cpp_tree_time, 
            cpp_nodes=cpp_tree_nodes, 
            py_time=py_tree_time,
            py_nodes=py_tree_nodes,
            title="Random tree")
plot_timing(cpp_ax, cpp_time=cpp_tree_time, cpp_nodes=cpp_tree_nodes, title="Random tree")
plot_timing(py_ax, py_time=py_tree_time, py_nodes=py_tree_nodes, title="Random tree")